<a href="https://colab.research.google.com/github/Meetra21/Contour__Gradient_pipeline/blob/main/Contour__Gradient_pipeline_for_Digits_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# If you need these in Colab, uncomment:
# !pip install --quiet tensorflow tensorflow-datasets opencv-python tqdm scikit-learn

import cv2
import time
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split

# -----------------------------
# 1) Boundary + central-diff gradients (+ magnitude)
# -----------------------------
def _auto_canny_edges_uint8(img_uint8, sigma=0.33):
    v = np.median(img_uint8)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    return cv2.Canny(img_uint8, lower, upper, L2gradient=True)

def compute_boundary_gradients(images, use_canny=True, add_mag=True):
    """
    images: (N, 28, 28) uint8 or float
    Returns: (N, C, 28, 28) float32 where C=2 (gx,gy) or 3 (gx,gy,mag)
    - central-difference grads (vectorized)
    - masked to edges (Canny) to emphasize contours
    - per-image L2 normalization to stabilize scale
    """
    imgs = images.astype(np.float32)
    if imgs.max() > 1.0:
        imgs /= 255.0

    N, H, W = imgs.shape
    p = np.pad(imgs, ((0,0),(1,1),(1,1)), mode='edge')

    gx = (p[:, 1:-1, 2:] - p[:, 1:-1, :-2]) * 0.5
    gy = (p[:, 2:, 1:-1] - p[:, :-2, 1:-1]) * 0.5

    if use_canny:
        edges = np.empty_like(images, dtype=np.uint8)
        imgs_u8 = (imgs * 255.0 + 0.5).astype(np.uint8)
        for i in range(N):
            edges[i] = _auto_canny_edges_uint8(imgs_u8[i])
        mask = (edges > 0).astype(np.float32)
    else:
        # fallback boundary proxy (morph gradient)
        kernel = np.ones((3,3), np.uint8)
        imgs_u8 = (imgs * 255.0 + 0.5).astype(np.uint8)
        dil = np.stack([cv2.dilate(im, kernel, iterations=1) for im in imgs_u8], axis=0)
        ero = np.stack([cv2.erode(im, kernel, iterations=1) for im in imgs_u8], axis=0)
        mask = ((dil - ero) > 0).astype(np.float32)

    gx *= mask
    gy *= mask

    eps = 1e-6
    norm = np.sqrt((gx**2 + gy**2).sum(axis=(1,2), keepdims=True)) + eps
    gx /= norm
    gy /= norm

    if add_mag:
        mag = np.sqrt(gx**2 + gy**2)
        feats = np.stack([gx, gy, mag], axis=1).astype(np.float32)  # (N,3,H,W)
    else:
        feats = np.stack([gx, gy], axis=1).astype(np.float32)       # (N,2,H,W)

    return feats

# -----------------------------
# 2) Load MNIST via TFDS
# -----------------------------
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

# -----------------------------
# 3) Convert to NumPy arrays
# -----------------------------
all_images, all_labels = [], []
for img, lbl in tfds.as_numpy(ds_train.concatenate(ds_test)):
    all_images.append(img.squeeze())   # (28,28) uint8
    all_labels.append(int(lbl))        # 0..9
all_images = np.stack(all_images)      # (70000,28,28)
all_labels = np.array(all_labels, dtype=np.int32)

# -----------------------------
# 4) Stratified 80/20 split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    all_images, all_labels,
    train_size=0.8,
    stratify=all_labels,
    random_state=42
)

# -----------------------------
# 5) Boundary+central-diff grads (+mag)
# -----------------------------
grad_train = compute_boundary_gradients(X_train, use_canny=True, add_mag=True)  # (N,3,28,28)
grad_test  = compute_boundary_gradients(X_test,  use_canny=True, add_mag=True)

# -----------------------------
# 6) Flatten + global z-score standardization
# -----------------------------
C = grad_train.shape[1]
X_train_feat = grad_train.reshape(-1, C*28*28)
X_test_feat  = grad_test.reshape(-1,  C*28*28)

mean = X_train_feat.mean(axis=0, keepdims=True)
std  = X_train_feat.std(axis=0, keepdims=True) + 1e-6
X_train_feat = (X_train_feat - mean) / std
X_test_feat  = (X_test_feat  - mean) / std

# -----------------------------
# 7) MLP model (accuracy-leaning but fast)
# -----------------------------
model = models.Sequential([
    layers.Input(shape=(C*28*28,)),
    layers.Dense(1024), layers.BatchNormalization(), layers.LeakyReLU(alpha=0.01),
    layers.Dropout(0.4),

    layers.Dense(512), layers.BatchNormalization(), layers.LeakyReLU(alpha=0.01),
    layers.Dropout(0.3),

    layers.Dense(256), layers.BatchNormalization(), layers.LeakyReLU(alpha=0.01),
    layers.Dropout(0.2),

    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# 8) Callbacks
# -----------------------------
es = callbacks.EarlyStopping(
    monitor='val_accuracy', patience=6,
    restore_best_weights=True, verbose=1
)
rlr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5,
    patience=3, min_lr=1e-6, verbose=1
)

# -----------------------------
# 9) Train
# -----------------------------
t0 = time.time()
history = model.fit(
    X_train_feat, y_train,
    validation_split=0.1,
    epochs=60,
    batch_size=256,                 # larger batch; features are small
    callbacks=[es, rlr],
    verbose=2
)
train_time = time.time() - t0

# -----------------------------
# 10) Evaluate
# -----------------------------
t1 = time.time()
test_loss, test_acc = model.evaluate(X_test_feat, y_test, verbose=0)
infer_time = time.time() - t1

# -----------------------------
# 11) Report
# -----------------------------
print(f"\nMNIST Boundary-Masked Central-Gradient (+Mag) MLP")
print(f"Input Channels  : {C}")
print(f"Test Accuracy   : {test_acc*100:.2f}%")
print(f"Training Time   : {train_time:.1f}s")
print(f"Inference Time  : {infer_time:.3f}s for {X_test_feat.shape[0]} samples")
